# Examples for the LPLM Interferometer Module

The `lplm_interferometer` module provides functions to decompose arbitrary linear transformations on $N$ optical modes into a sequence of $6N+1$ diagonal phases masks interleaved with discrete Fourier transforms (DFT). The method is based on a paper published by López Pastor, Lundeen and Marquardt in 2021 [1]. 

### References
1. Víctor López Pastor, Jeff Lundeen, and Florian Marquardt, "Arbitrary optical wave evolution with Fourier transforms and phase masks," Opt. Express 29, 38441-38450 (2021)


In [11]:
# Import dependencies
from unitary_decomp import lplm_interferometer as li

import numpy as np
from scipy.stats import unitary_group

We start by generating a random unitary matrix $U$ of size $N \times N$.

In [12]:
N = 6 # Dimension of the unitary matrix

# Generate a random unitary matrix of dimension N
U = unitary_group(dim=N, seed=43).rvs()

# Print the unitary matrix
print("Unitary matrix U:\n\n", U.round(3))

Unitary matrix U:

 [[ 0.067-0.529j -0.026+0.35j  -0.304-0.417j  0.011+0.23j  -0.356+0.08j
   0.358-0.11j ]
 [ 0.129+0.214j  0.448+0.653j  0.323-0.047j -0.258-0.133j  0.144+0.306j
   0.071-0.008j]
 [-0.431-0.336j -0.125+0.071j  0.33 -0.365j -0.101-0.371j  0.249-0.284j
  -0.191-0.334j]
 [-0.271-0.384j -0.102+0.j     0.099+0.377j -0.599+0.38j   0.174+0.067j
   0.017+0.281j]
 [ 0.014+0.001j  0.23 +0.185j -0.174+0.412j -0.062-0.197j -0.029-0.709j
   0.396-0.096j]
 [-0.364+0.029j  0.172-0.319j  0.095-0.154j -0.03 -0.41j  -0.17 +0.207j
   0.519+0.438j]]


We now apply the `lplm_decomposition` function found in the `lplm_interferometer` module to compute all the phase masks that decompose the matrix $U$. The function returns a `LplmDecomp` object that has two attributes:

- `D`: Output phase mask of the system
- `mask_sequence`: List of the $6N$ phase masks that are interleaved by Fourier transform matrices.

These phase masks allow for a full description of the matrix $U$:

$U = D \Pi_{i=1}^{6N}FM_i$,

where $M_i \in \text{mask\_sequence}\ \forall i \in {1, ..., 6N}$. 
$F$ is the discrete Fourier tranform matrix, whose matrix elements are given by 

$F_{j, k} = \frac{1}{\sqrt{N}}e^{-i2\pi jk/N}$.

In [13]:
# Perform the LPLM decomposition
decomposition = li.lplm_decomposition(U)

# Output type of the lplm_decomposition function
print("Output type:", type(decomposition).__name__, end="\n\n")

# Circuit length of the mask_sequence
print("Circuit lenght:", len(decomposition.mask_sequence))

# Assert that we get 6N masks
assert 6*N == len(decomposition.mask_sequence)

Output type: LplmDecomp

Circuit lenght: 36


The `lplm_interferometer` module also provides a function to reconstruct the unitary matrix $U$ given its decomposition. This allows to verify the fidelity of the decomposition.

In [14]:
# Reconstruct the unitary matrix from the decomposition
reconstructed_U = li.circuit_reconstruction(decomposition)

# Print the reconstructed matrix
print("Reconstructed matrix U:\n\n", reconstructed_U.round(3))

# Check if the reconstructed matrix matches the original unitary matrix
assert np.allclose(U, reconstructed_U)

Reconstructed matrix U:

 [[ 0.067-0.529j -0.026+0.35j  -0.304-0.417j  0.011+0.23j  -0.356+0.08j
   0.358-0.11j ]
 [ 0.129+0.214j  0.448+0.653j  0.323-0.047j -0.258-0.133j  0.144+0.306j
   0.071-0.008j]
 [-0.431-0.336j -0.125+0.071j  0.33 -0.365j -0.101-0.371j  0.249-0.284j
  -0.191-0.334j]
 [-0.271-0.384j -0.102+0.j     0.099+0.377j -0.599+0.38j   0.174+0.067j
   0.017+0.281j]
 [ 0.014+0.001j  0.23 +0.185j -0.174+0.412j -0.062-0.197j -0.029-0.709j
   0.396-0.096j]
 [-0.364+0.029j  0.172-0.319j  0.095-0.154j -0.03 -0.41j  -0.17 +0.207j
   0.519+0.438j]]
